# Linear SVM Classification

SVM classifier as fitting the widest possible street(represented by the parallel dashed lines) between the classes. This is called large margin classification.

**Notice**<br>
adding more training instances "off the street" will not affect the decision boundary at all: it is fully determined(or "supported") by the instances located on the edge of the street. These instances are called the *support vector*

**SVMs are sensitice to the features scales**.

# Soft Margin Classification